In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report

print("CS4921 Lab 2 - Alexander Huang")
print("Library Versions:")
print('numpy:',np.__version__)
print('pandas:',pd.__version__)
print('tensorflow:',tf.__version__)
print('tensorflow probability:', tfp.__version__)

CS4921 Lab 2 - Alexander Huang
Library Versions:
numpy: 1.22.4
pandas: 1.5.2
tensorflow: 2.6.0
tensorflow probability: 0.14.0


In [2]:
# number of epochs
n_linear_epochs = 300
n_nn_epochs = 300
n_ion_epochs = 300
verbose_option = True

# California Housing Models

In [3]:
# Import Dataset
cali_data = pd.read_csv('cal_housing.csv')
air_data = pd.read_table('airfoil_self_noise.dat', header=None)

# Data Exploration
labels = {'longitude: continuous.': 'longitude',
          'latitude: continuous.': 'latitude',
          'housingMedianAge: continuous. ': 'housingMedianAge',
          'totalRooms: continuous. ': 'totalRooms',
          'totalBedrooms: continuous. ': 'totalBedrooms',
          'population: continuous. ': 'population',
          'households: continuous. ': 'households',
          'medianIncome: continuous. ': 'medianIncome',
          'medianHouseValue: continuous. ': 'medianHouseValue'
         }
# Change the columns to something reasonable
cali_data = cali_data.rename(columns=labels)

cali_data.info()
print('Null values')
print(cali_data.isnull().sum())
print(cali_data.head())
print(cali_data.describe())

# feature engineering 8 -> 7 features
#cali_data['numRooms'] = cali_data['totalRooms']/cali_data['households']
#cali_data['numBedrooms'] = cali_data['totalBedrooms']/cali_data['households']
#cali_data['personsPerHouse'] = cali_data['population']/cali_data['households']
#cali_data = cali_data.drop(['totalRooms', 'totalBedrooms', 'population', 'households'], axis=1)

#air_data.info()
# Split features and labels
y = cali_data['medianHouseValue'].to_frame()
x = cali_data.drop(['medianHouseValue'], axis=1)

#y = air_data.iloc[:,5]
#x = air_data.iloc[:,0:5]
print('Features')
print(x)
print('Target')
print(y)


y_mu=y.to_numpy().mean()
y_sigma=y.to_numpy().std()
y_orig = y.to_numpy().flatten()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   longitude         20640 non-null  float64
 1   latitude          20640 non-null  float64
 2   housingMedianAge  20640 non-null  int64  
 3   totalRooms        20640 non-null  int64  
 4   totalBedrooms     20640 non-null  int64  
 5   population        20640 non-null  int64  
 6   households        20640 non-null  int64  
 7   medianIncome      20640 non-null  float64
 8   medianHouseValue  20640 non-null  int64  
dtypes: float64(3), int64(6)
memory usage: 1.4 MB
Null values
longitude           0
latitude            0
housingMedianAge    0
totalRooms          0
totalBedrooms       0
population          0
households          0
medianIncome        0
medianHouseValue    0
dtype: int64
   longitude  latitude  housingMedianAge  totalRooms  totalBedrooms  \
0    -122.23     37.88         

In [4]:
# Scaling
scale = lambda y: (y-y_mu)/y_sigma
unscale = lambda y: y_sigma*y+y_mu

x = scale(x.to_numpy())
y = scale(y.to_numpy())
print('x:', x)
print(x.shape)
print('y:', y)
print(y.shape)

x: [[-1.79368219 -1.79229467 -1.79226763 ... -1.78983247 -1.79153102
  -1.79255079]
 [-1.7936821  -1.79229484 -1.79244095 ... -1.77181575 -1.78276097
  -1.792551  ]
 [-1.79368228 -1.79229493 -1.7921723  ... -1.78832458 -1.79108905
  -1.79256005]
 ...
 [-1.79367344 -1.79228124 -1.79247562 ... -1.78389623 -1.78887054
  -1.79260821]
 [-1.7936743  -1.79228124 -1.79246695 ... -1.7862014  -1.78959849
  -1.79260676]
 [-1.79367361 -1.79228176 -1.79248428 ... -1.78060313 -1.78802993
  -1.79260224]]
(20640, 8)
y: [[ 2.12963148]
 [ 1.31415614]
 [ 1.25869341]
 ...
 [-0.99274649]
 [-1.05860847]
 [-1.01787803]]
(20640, 1)


In [5]:
n_examples = y.shape[0]
expectNLL = lambda y, rv_y: -rv_y.log_prob(y)/n_examples

<span>1.</span>Using Pytorch or Tensoflow, perform Maximum Likelihood Estimation (MLE) for a linear Gaussian
prediction model with homoscedastic uncertainty for the California housing price dataset.

In [74]:
input = tf.keras.layers.Input(8)
linear = tf.keras.layers.Dense(1)(input)
s = tfp.layers.VariableLayer(shape=[1], activation=tf.keras.activations.exponential)(True)
p = tfp.layers.DistributionLambda(lambda t: tfp.distributions.Normal(loc=t[0], scale=t[1]))([linear,s])
model = tf.keras.Model(inputs=input, outputs=p, name="HousingLinearModel")

model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01), loss=expectNLL)
print(model.summary())

Model: "HousingLinearModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        [(None, 8)]               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
_________________________________________________________________
distribution_lambda_1 (Distr multiple                  0         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
# Training
model.fit(x, y, epochs=n_linear_epochs, verbose=verbose_option)

Epoch 1/300


2023-01-27 05:47:35.085563: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


645/645 [==============================] - 1s 1ms/step - loss: 7.1127e-05
Epoch 2/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8895e-05
Epoch 3/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8949e-05
Epoch 4/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8967e-05
Epoch 5/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8898e-05
Epoch 6/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8937e-05
Epoch 7/300
645/645 [==============================] - 1s 1ms/step - loss: 6.9006e-05
Epoch 8/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8943e-05
Epoch 9/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8976e-05
Epoch 10/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8859e-05
Epoch 11/300
645/645 [==============================] - 1s 1ms/step - loss: 6.9040e-05
Epoch 12/300
645/645 [==============================] - 1s 865us

645/645 [==============================] - 1s 1ms/step - loss: 6.8662e-05
Epoch 96/300
645/645 [==============================] - 1s 937us/step - loss: 6.8615e-05
Epoch 97/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8635e-05
Epoch 98/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8703e-05
Epoch 99/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8706e-05
Epoch 100/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8698e-05
Epoch 101/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8652e-05
Epoch 102/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8688e-05
Epoch 103/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8640e-05
Epoch 104/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8717e-05
Epoch 105/300
645/645 [==============================] - 1s 936us/step - loss: 6.8643e-05
Epoch 106/300
645/645 [=======================

645/645 [==============================] - 1s 1ms/step - loss: 6.8337e-05
Epoch 280/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8315e-05
Epoch 281/300
645/645 [==============================] - 1s 961us/step - loss: 6.8408e-05
Epoch 282/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8376e-05
Epoch 283/300
645/645 [==============================] - 1s 950us/step - loss: 6.8387e-05
Epoch 284/300
645/645 [==============================] - 1s 955us/step - loss: 6.8376e-05
Epoch 285/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8362e-05
Epoch 286/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8342e-05
Epoch 287/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8345e-05
Epoch 288/300
645/645 [==============================] - 1s 2ms/step - loss: 6.8414e-05
Epoch 289/300
645/645 [==============================] - 1s 1ms/step - loss: 6.8356e-05
Epoch 290/300
645/645 [=================

In [8]:
y_dist = model(x)
print('Model mean:', y_dist.mean())
print('Model standard deviation:', y_dist.stddev())

Model mean: tf.Tensor(
[[-0.22122586]
 [ 0.07803237]
 [-0.1918155 ]
 ...
 [-0.16441166]
 [-0.17819703]
 [-0.14659417]], shape=(20640, 1), dtype=float32)
Model standard deviation: tf.Tensor(
[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]], shape=(20640, 1), dtype=float32)


In [9]:
yhat1 = model.predict(x)
y_pred1 = unscale(yhat1)
y_pred1 = y_pred1.flatten()
for i in range(10):
    print('Predictions:', y_pred1[i])
    print('True:', y_orig[i])
print('RMSE:', np.sqrt(mean_squared_error(y_orig, y_pred1)))

Predictions: 107172.695
True: 452600
Predictions: 201404.1
True: 358500
Predictions: 292299.3
True: 352100
Predictions: 147564.3
True: 341300
Predictions: 257415.05
True: 342200
Predictions: 470251.0
True: 269700
Predictions: 218970.72
True: 299200
Predictions: 86816.664
True: 241400
Predictions: 152812.72
True: 226700
Predictions: 267716.56
True: 261100
RMSE: 162999.93789051851


In [10]:
input2 = tf.keras.layers.Input(8)
h1 = tf.keras.layers.Dense(50, activation=tf.keras.activations.relu, name="hidden1")(input2)
h2 = tf.keras.layers.Dense(50, activation=tf.keras.activations.relu, name="hidden2")(h1)
h3 = tf.keras.layers.Dense(20, activation=tf.keras.activations.relu, name="hidden3")(h2)
m2 = tf.keras.layers.Dense(1, name="mu")(h3)
s2 = tfp.layers.VariableLayer(shape=[1], activation=tf.keras.activations.exponential, name="sigma")(h3)
p2 = tfp.layers.DistributionLambda(lambda t: tfp.distributions.Normal(loc=t[0], scale=t[1]), name="output")([m2, s2])
model2 = tf.keras.Model(inputs=input2, outputs=p2, name="HousingNNModel")

model2.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), loss=expectNLL)
print(model2.summary())

Model: "HousingNNModel"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
hidden1 (Dense)                 (None, 50)           450         input_2[0][0]                    
__________________________________________________________________________________________________
hidden2 (Dense)                 (None, 50)           2550        hidden1[0][0]                    
__________________________________________________________________________________________________
hidden3 (Dense)                 (None, 20)           1020        hidden2[0][0]                    
_____________________________________________________________________________________

In [11]:
# Training
model2.fit(x, y, epochs=100, verbose=verbose_option)

Epoch 1/100
645/645 [==============================] - 2s 2ms/step - loss: 6.8946e-05
Epoch 2/100
645/645 [==============================] - 1s 1ms/step - loss: 6.8786e-05
Epoch 3/100
645/645 [==============================] - 1s 1ms/step - loss: 6.8785e-05
Epoch 4/100
645/645 [==============================] - 1s 1ms/step - loss: 6.8783e-05
Epoch 5/100
645/645 [==============================] - 1s 1ms/step - loss: 6.8755e-05
Epoch 6/100
645/645 [==============================] - 1s 1ms/step - loss: 6.8757e-05
Epoch 7/100
645/645 [==============================] - 1s 1ms/step - loss: 6.8757e-05
Epoch 8/100
645/645 [==============================] - 1s 1ms/step - loss: 6.8764e-05
Epoch 9/100
645/645 [==============================] - 1s 1ms/step - loss: 6.8758e-05
Epoch 10/100
645/645 [==============================] - 1s 1ms/step - loss: 6.8758e-05
Epoch 11/100
645/645 [==============================] - 1s 1ms/step - loss: 6.8747e-05
Epoch 12/100
645/645 [==============================

In [12]:
y_dist2 = model2(x)
print('Model mean:', y_dist2.mean())
print('Model standard deviation:', y_dist2.stddev())

Model mean: tf.Tensor(
[[0.00164472]
 [0.00164472]
 [0.00164472]
 ...
 [0.00164472]
 [0.00164472]
 [0.00164472]], shape=(20640, 1), dtype=float32)
Model standard deviation: tf.Tensor(
[[1.0023664]
 [1.0023664]
 [1.0023664]
 ...
 [1.0023664]
 [1.0023664]
 [1.0023664]], shape=(20640, 1), dtype=float32)


In [13]:
yhat2 = model.predict(x)
y_pred2 = unscale(yhat2)
y_pred2 = y_pred2.flatten()
for i in range(10):
    print('Prediction', i, ':', y_pred2[i])
    print('True', i, ':', y_orig[i])
print('RMSE:', np.sqrt(mean_squared_error(y_orig, y_pred2)))

Prediction 0 : 196625.5
True 0 : 452600
Prediction 1 : 121910.95
True 1 : 358500
Prediction 2 : 410406.5
True 2 : 352100
Prediction 3 : 247495.16
True 3 : 341300
Prediction 4 : 169650.44
True 4 : 342200
Prediction 5 : 184892.45
True 5 : 269700
Prediction 6 : 153600.25
True 6 : 299200
Prediction 7 : 260198.66
True 7 : 241400
Prediction 8 : 203221.55
True 8 : 226700
Prediction 9 : 244312.89
True 9 : 261100
RMSE: 163641.9845462217


# Ionosphere dataset models

In [14]:
ion_data = pd.read_csv('ionosphere.csv', header=None)
print(ion_data.info)
print('Null values')
print(ion_data.isnull().sum())

<bound method DataFrame.info of      0   1        2        3        4        5        6        7        8   \
0     1   0  0.99539 -0.05889  0.85243  0.02306  0.83398 -0.37708  1.00000   
1     1   0  1.00000 -0.18829  0.93035 -0.36156 -0.10868 -0.93597  1.00000   
2     1   0  1.00000 -0.03365  1.00000  0.00485  1.00000 -0.12062  0.88965   
3     1   0  1.00000 -0.45161  1.00000  1.00000  0.71216 -1.00000  0.00000   
4     1   0  1.00000 -0.02401  0.94140  0.06531  0.92106 -0.23255  0.77152   
..   ..  ..      ...      ...      ...      ...      ...      ...      ...   
346   1   0  0.83508  0.08298  0.73739 -0.14706  0.84349 -0.05567  0.90441   
347   1   0  0.95113  0.00419  0.95183 -0.02723  0.93438 -0.01920  0.94590   
348   1   0  0.94701 -0.00034  0.93207 -0.03227  0.95177 -0.03431  0.95584   
349   1   0  0.90608 -0.01657  0.98122 -0.01989  0.95691 -0.03646  0.85746   
350   1   0  0.84710  0.13533  0.73638 -0.06151  0.87873  0.08260  0.88928   

          9   ...       25     

In [15]:
"""
# One hot, label is last two columns
def one_hot(cat, hot):
    if cat == hot:
        return int(1)
    else:
        return int(0)
    
ion_data[35] = ion_data[34].apply(one_hot, hot='g')
ion_data[36] = ion_data[34].apply(one_hot, hot='b')
"""

def ordinal_encoder(category):
    dict = {'g': 0, 'b': 1}
    return dict[category]

print('g class:', ordinal_encoder('g'))
print('b class:', ordinal_encoder('b'))
ion_data[34] = ion_data[34].apply(ordinal_encoder)
print('After ordinal encoding:')
print(ion_data[34].to_numpy())


g class: 0
b class: 1
After ordinal encoding:
[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [16]:
# split x and y
x = ion_data.iloc[:,0:33].to_numpy()
y = ion_data.iloc[:,34].to_numpy().reshape(-1,1)

# Scaling
scaler = StandardScaler()
x = scaler.fit_transform(x)
print(x.shape)
print(y.shape)

(351, 33)
(351, 1)


<span>3.</span>Using Pytorch or Tensoflow, perform MLE for a neural network Bernoulli prediction model for the
ionosphere dataset.

In [17]:
input3 = tf.keras.layers.Input(33)
h1 = tf.keras.layers.Dense(5, activation=tf.keras.activations.relu, name="hidden1")(input3)
out3 = tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, name="out")(h1)

model3 = tf.keras.Model(inputs=input3, outputs=out3, name="IonosphereNNMLE")

model3.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01), loss=tf.losses.binary_crossentropy, metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5)])
print(model3.summary())

Model: "IonosphereNNMLE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 33)]              0         
_________________________________________________________________
hidden1 (Dense)              (None, 5)                 170       
_________________________________________________________________
out (Dense)                  (None, 1)                 6         
Total params: 176
Trainable params: 176
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
model3.fit(x, y, epochs=30, verbose=True)

Epoch 1/30
11/11 [==============================] - 0s 1ms/step - loss: 0.5941 - binary_accuracy: 0.7123
Epoch 2/30
11/11 [==============================] - 0s 2ms/step - loss: 0.4802 - binary_accuracy: 0.8205
Epoch 3/30
11/11 [==============================] - 0s 2ms/step - loss: 0.4117 - binary_accuracy: 0.8519
Epoch 4/30
11/11 [==============================] - 0s 1ms/step - loss: 0.3509 - binary_accuracy: 0.8689
Epoch 5/30
11/11 [==============================] - 0s 1ms/step - loss: 0.3108 - binary_accuracy: 0.8917
Epoch 6/30
11/11 [==============================] - 0s 1ms/step - loss: 0.2762 - binary_accuracy: 0.9003
Epoch 7/30
11/11 [==============================] - 0s 1ms/step - loss: 0.2578 - binary_accuracy: 0.9031
Epoch 8/30
11/11 [==============================] - 0s 1ms/step - loss: 0.2399 - binary_accuracy: 0.9117
Epoch 9/30
11/11 [==============================] - 0s 1ms/step - loss: 0.2253 - binary_accuracy: 0.9174
Epoch 10/30
11/11 [==============================] - 0s

In [19]:
yhat = model3.predict(x)
def labelmaker(prob):
    if prob > 0.4:
        return 1
    else:
        return 0

yhat = np.apply_along_axis(labelmaker, 1, yhat)
print(classification_report(y.flatten(), yhat))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       225
           1       0.96      0.96      0.96       126

    accuracy                           0.97       351
   macro avg       0.97      0.97      0.97       351
weighted avg       0.97      0.97      0.97       351



<span>4.</span>Using Pytorch or Tensoflow, perform Maximum a Posterori (MAP) estimation for a neural network
Bernoulli prediction model and a mean zero Gaussian parameter prior for the ionosphere dataset.

In [20]:
input4 = tf.keras.layers.Input(33)
h1 = tf.keras.layers.Dense(5, activation=tf.keras.activations.relu, name="hidden1")(input4)
reg4 = tf.keras.layers.Dense(5, kernel_regularizer=tf.keras.regularizers.L2(l2=0.05), name="L2Regularization")(h1)
out4 = tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid)(reg4)

model4 = tf.keras.Model(inputs=input4, outputs=out4, name="IonosphereNNMAP")

model4.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), loss=tf.losses.BinaryCrossentropy())
print(model4.summary())

Model: "IonosphereNNMAP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 33)]              0         
_________________________________________________________________
hidden1 (Dense)              (None, 5)                 170       
_________________________________________________________________
L2Regularization (Dense)     (None, 5)                 30        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6         
Total params: 206
Trainable params: 206
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
model4.fit(x, y, epochs=50, verbose=True)

Epoch 1/50
11/11 [==============================] - 0s 2ms/step - loss: 0.9757
Epoch 2/50
11/11 [==============================] - 0s 1ms/step - loss: 0.9328
Epoch 3/50
11/11 [==============================] - 0s 1ms/step - loss: 0.8974
Epoch 4/50
11/11 [==============================] - 0s 1ms/step - loss: 0.8651
Epoch 5/50
11/11 [==============================] - 0s 1ms/step - loss: 0.8366
Epoch 6/50
11/11 [==============================] - 0s 1ms/step - loss: 0.8093
Epoch 7/50
11/11 [==============================] - 0s 1ms/step - loss: 0.7840
Epoch 8/50
11/11 [==============================] - 0s 1ms/step - loss: 0.7589
Epoch 9/50
11/11 [==============================] - 0s 1ms/step - loss: 0.7344
Epoch 10/50
11/11 [==============================] - 0s 1ms/step - loss: 0.7110
Epoch 11/50
11/11 [==============================] - 0s 1ms/step - loss: 0.6900
Epoch 12/50
11/11 [==============================] - 0s 1ms/step - loss: 0.6698
Epoch 13/50
11/11 [==============================

In [22]:
yhat = model4.predict(x)
def labelmaker(prob):
    if prob > 0.4:
        return 1
    else:
        return 0

yhat = np.apply_along_axis(labelmaker, 1, yhat)
print(classification_report(y.flatten(), yhat))

              precision    recall  f1-score   support

           0       0.93      0.94      0.94       225
           1       0.90      0.88      0.89       126

    accuracy                           0.92       351
   macro avg       0.91      0.91      0.91       351
weighted avg       0.92      0.92      0.92       351



# Test with Bernoullli distribution object instead of BCE loss

In [70]:
input31 = tf.keras.layers.Input(33)
h31 = tf.keras.layers.Dense(10, activation=tf.keras.activations.relu, name="hidden1")(input31)
h32 = tf.keras.layers.Dense(10, activation=tf.keras.activations.relu, name="hidden2")(h31)
h33 = tf.keras.layers.Dense(10, activation=tf.keras.activations.relu, name="hidden3")(h32)
p_val = tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, name="p_val")(h33)
out31 = tfp.layers.DistributionLambda(lambda t: tfp.distributions.Bernoulli(probs=t,  dtype=tf.float64), name="output")(p_val)

model31 = tf.keras.Model(inputs=input31, outputs=out31, name="IonosphereNNMLEwDistr")

model31.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), loss=expectNLL)
print(model31.summary())

Model: "IonosphereNNMLEwDistr"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 33)]              0         
_________________________________________________________________
hidden1 (Dense)              (None, 10)                340       
_________________________________________________________________
hidden2 (Dense)              (None, 10)                110       
_________________________________________________________________
hidden3 (Dense)              (None, 10)                110       
_________________________________________________________________
p_val (Dense)                (None, 1)                 11        
_________________________________________________________________
output (DistributionLambda)  multiple                  0         
Total params: 571
Trainable params: 571
Non-trainable params: 0
_______________________________________________

In [71]:
model31.fit(x, y, epochs=50, verbose=True)

Epoch 1/50
11/11 [==============================] - 0s 2ms/step - loss: 3.2111e-05
Epoch 2/50
11/11 [==============================] - 0s 2ms/step - loss: 3.1104e-05
Epoch 3/50
11/11 [==============================] - 0s 2ms/step - loss: 3.0065e-05
Epoch 4/50
11/11 [==============================] - 0s 2ms/step - loss: 2.9196e-05
Epoch 5/50
11/11 [==============================] - 0s 2ms/step - loss: 2.8392e-05
Epoch 6/50
11/11 [==============================] - 0s 2ms/step - loss: 2.7695e-05
Epoch 7/50
11/11 [==============================] - 0s 2ms/step - loss: 2.7080e-05
Epoch 8/50
11/11 [==============================] - 0s 2ms/step - loss: 2.6449e-05
Epoch 9/50
11/11 [==============================] - 0s 2ms/step - loss: 2.5879e-05
Epoch 10/50
11/11 [==============================] - 0s 2ms/step - loss: 2.5266e-05
Epoch 11/50
11/11 [==============================] - 0s 1ms/step - loss: 2.4646e-05
Epoch 12/50
11/11 [==============================] - 0s 1ms/step - loss: 2.4036e-05
E

In [73]:
yhat = model31.predict(x)
yhat = np.apply_along_axis(labelmaker, 1, yhat)
print(classification_report(y.flatten(), yhat.flatten()))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94       225
           1       0.91      0.87      0.89       126

    accuracy                           0.92       351
   macro avg       0.92      0.91      0.91       351
weighted avg       0.92      0.92      0.92       351

